In [52]:
import os
import re

# Change directory to the root of the SpatialDeconBenchmark repository
os.chdir('/'.join(re.match(r'^(.*SpatialDeconBenchmark)', os.getcwd()).group(0).split('/')))

In [53]:
import numpy as np
import pandas as pd 
from libpysal.weights import W
from scipy.spatial import KDTree
from esda.moran import Moran, Moran_Local
from scipy.stats import pearsonr

np.random.seed(42)


class SpotMetrics:
    def __init__(self, deconvolved_mtx, st_counts_mtx, st_locations_mtx, _marker_genes_dict):
        """
        Initialize the SpotMetrics class with matrices and marker gene data.

        Parameters:
        - deconvolved_mtx: DataFrame, spot by celltype matrix.
        - st_counts_mtx: DataFrame, gene by spot matrix.
        - st_locations_mtx: DataFrame, spot by coordinate matrix.
        - marker_genes_dict: dict, mapping cell types to marker genes.
        """
        self.deconvolved_mtx = deconvolved_mtx
        self.st_counts_mtx = st_counts_mtx
        self.st_locations_mtx = st_locations_mtx
        self.marker_genes_dict = _marker_genes_dict
        self.pcc = None
        self.spatial_weights = None
        self.global_moran = None
        self.local_moran = None

    def check_mtx_format(self):
        """
        Check if matrix formats align correctly across provided dataframes.
        Raises ValueError if mismatches are found.
        """
        if set(self.st_counts_mtx.columns) != set(self.st_locations_mtx.index):
            raise ValueError("Mismatch in spot labels between counts and location matrices.")

        if set(self.st_counts_mtx.columns) != set(self.deconvolved_mtx.index):
            print(set(self.st_counts_mtx.columns[:10]))
            print(set(self.deconvolved_mtx.index[:10]))
            raise ValueError("Mismatch in spot labels between counts and deconvolved matrices.")

        # Subset matrices to ensure alignment
        self.deconvolved_mtx = self.deconvolved_mtx.loc[self.st_counts_mtx.columns]
        self.st_locations_mtx = self.st_locations_mtx.loc[self.st_counts_mtx.columns]

    def calculate_pcc(self):
        """
        Calculate Pearson's Correlation Coefficient between estimated cell type abundances
        and expressions of canonical marker genes. Results are stored in a dictionary.
        """
        cell_types_pcc = {}

        for cell_type, markers in self.marker_genes_dict.items():
            marker_pccs = {}
            for marker in markers:
                # Extract gene expression and cell type abundance
                gene_expression = self.st_counts_mtx.loc[marker]
                cell_abundance = self.deconvolved_mtx[cell_type]
                
                # Calculate Pearson correlation
                pcc, pcc_p_value = pearsonr(gene_expression, cell_abundance)
                marker_pccs[marker] = (pcc, pcc_p_value)

            cell_types_pcc[cell_type] = marker_pccs
        
        self.pcc = cell_types_pcc

    def compute_spatial_weights(self, k):
        """
        Compute spatial weights for each spot based on k nearest neighbors.
        """
        # Extract coordinates and build KDTree
        xy_coords = self.st_locations_mtx[['x', 'y']].values
        tree = KDTree(xy_coords)

        weights = {}
        for i, coord in enumerate(xy_coords):
            distances, indices = tree.query(coord, k=k+1)  # Include self in query
            neighbors = indices[1:]  # Exclude the spot itself
            weights[i] = {n: 1 for n in neighbors}  # Uniform weight of 1 for each neighbor

        self.spatial_weights = W(weights)

    def calculate_moran(self):
        """
        Compute Global and Local Moran's I statistics for cell type distributions.
        Requires spatial weights to be pre-computed.
        """
        if not hasattr(self, 'spatial_weights'):
            raise ValueError("Spatial weights must be calculated before Moran's I.")

        global_moran = {}
        local_moran = {}

        for cell_type in self.deconvolved_mtx.columns:
            values = self.deconvolved_mtx[cell_type]
            w = self.spatial_weights

            # Calculate global and local Moran's I
            moran_global = Moran(values, w)
            moran_local = Moran_Local(values, w)

            global_moran[cell_type] = (moran_global.I, moran_global.p_sim)
            local_moran[cell_type] = (moran_local.Is, moran_local.p_sim)

        self.global_moran = global_moran
        self.local_moran = local_moran

In [54]:
# Cell types and markers selected for visualization in benchmark paper
marker_genes_dict = {
    "Oligo": ["Mog", "Plp1"],
    "Ext_Thal": ["Synpo2", "Ptpn3", "Slc17a6"],
    "Ext_ClauPyr": ["Nr4a2", "Synpr"]
}

In [55]:
# Load the Visium deconvolved matrix
visium_spatial = pd.read_csv(os.getcwd() + "/data/Visium/filtered_st_counts.csv", index_col=0)
visium_locations = pd.read_csv(os.getcwd() + "/data/Visium/filtered_st_locations.csv", index_col=0)
visium_locations.index = list(map(lambda x: ".".join(x.split("-")), visium_locations.index))

results_dir = os.getcwd() + "/results/Visium/"

In [56]:
def run_benchmarking():
    benchmark_pcc = []
    benchmark_mi = []

    for result in os.listdir(results_dir):
        method = result.split("_")[1].split(".")[0]
        
        visium_deconvolved = pd.read_csv(results_dir + result, index_col=0)
        visium_deconvolved.index = list(map(lambda x: ".".join(x.split("-")), visium_deconvolved.index))
        
        visium_metrics = SpotMetrics(visium_deconvolved, visium_spatial, visium_locations, marker_genes_dict)
        visium_metrics.check_mtx_format()
        visium_metrics.calculate_pcc()
        visium_metrics.compute_spatial_weights(6)
        visium_metrics.calculate_moran()
        
        # Collect data for Pearson Correlation Coefficient
        for cell_type, pcc_values in visium_metrics.pcc.items():
            for marker, (pcc, pcc_p_value) in pcc_values.items():
                data_pcc = {
                    'method': method,
                    'cell_type': cell_type,
                    'marker': marker,
                    'PCC': pcc,
                    'PCC_p_value': pcc_p_value  
                }
                benchmark_pcc.append(pd.DataFrame([data_pcc]))

        # Collect data for Moran's I
        for cell_type in visium_metrics.global_moran:
            global_moran, moran_p_value = visium_metrics.global_moran[cell_type]
            data_mi = {
                'method': method,
                'cell_type': cell_type,
                'MI': global_moran,
                'MI_p_value': moran_p_value
            }
            benchmark_mi.append(pd.DataFrame([data_mi]))

    # Concatenate and save PCC benchmarking results
    if benchmark_pcc:
        benchmark_pcc_df = pd.concat(benchmark_pcc, ignore_index=True)
        benchmark_pcc_df.to_csv(os.getcwd() + "/results/Visium_PCC_Results.csv", index=False)

    # Concatenate and save MI benchmarking results
    if benchmark_mi:
        benchmark_mi_df = pd.concat(benchmark_mi, ignore_index=True)
        benchmark_mi_df.to_csv(os.getcwd() + "/results/Visium_MI_Results.csv", index=False)

# Run benchmarking
run_benchmarking()

C:\Users\Lem\anaconda3\envs\metrics-env\lib\site-packages\esda\moran.py:254: RuntimeWarning: invalid value encountered in scalar divide
  k = k_num / k_den
C:\Users\Lem\anaconda3\envs\metrics-env\lib\site-packages\esda\moran.py:266: RuntimeWarning: invalid value encountered in scalar divide
  return self.n / s0 * inum / self.z2ss
C:\Users\Lem\anaconda3\envs\metrics-env\lib\site-packages\esda\moran.py:223: RuntimeWarning: invalid value encountered in divide
  self.z /= sy
C:\Users\Lem\anaconda3\envs\metrics-env\lib\site-packages\esda\moran.py:254: RuntimeWarning: invalid value encountered in scalar divide
  k = k_num / k_den
C:\Users\Lem\anaconda3\envs\metrics-env\lib\site-packages\esda\moran.py:266: RuntimeWarning: invalid value encountered in scalar divide
  return self.n / s0 * inum / self.z2ss
C:\Users\Lem\anaconda3\envs\metrics-env\lib\site-packages\esda\moran.py:223: RuntimeWarning: invalid value encountered in divide
  self.z /= sy
